# 유방암 종양 예측

## 악성 종양인지, 양성 종양(정상)인지 예측

## 데이터셋 불러오기

* radius : 반경
* texture : 질감
* perimeter : 둘레
* area : 면적
* smoothness : 매끄러움
* compactness : 조그만 정도
* concavity : 윤곽의 오목함
* concave points : 윤곽의 오목한 부분의 수
* symmetry : 대칭
* fractal dimension : 프랙탈 차원
* target : Malignant-악성 종양 / Benign-양성 종양(정상)
---
* mean ~ : 데이터별 평균 값
* ~ error : 데이터별 표준편차 값
* worst ~ : 데이터별 가장 큰 3개의 평균으로 만든 값

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/DrKAI/dataset/main/breast_cancer.csv')

data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,Malignant
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,Malignant
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,Malignant
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,Malignant
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,Malignant


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

## 데이터 전처리

<details><summary>Hint</summary>

* x, y 구분 필요
* y에 대한 전처리 필요
* 데이터셋 구분 필요

</details>

In [35]:
# x, y 구분
x = data.drop('target', axis=1)
y = data['target']

In [36]:
# train_test_split
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2)

In [37]:
y_train.unique()

array(['Malignant', 'Benign'], dtype=object)

In [38]:
y_train.loc[y_train=='Benign'] = 0
y_train.loc[y_train=='Malignant'] = 1

In [53]:
y_test.loc[y_test=='Benign'] = 0
y_test.loc[y_test=='Malignant'] = 1

In [46]:
y_train.unique()

array([1, 0], dtype=object)

## Locally connected layer를 구성하기 위한 feature 설정

* mean, error, worst를 각 feature별로 묶어 새로운 변수를 만든다.
        ex) mean radius, radius error, worst radius

<details><summary> Hint </summary>

* x의 train 구조와 test 구조는 동일해야 한다.

</details>

In [40]:
x_train_radius = x_train.loc[:, ['mean radius', 'radius error', 'worst radius']]
x_train_texture = x_train.loc[:, ['mean texture', 'texture error', 'worst texture']]
x_train_perimeter = x_train.loc[:, ['mean perimeter', 'perimeter error', 'worst perimeter']]
x_train_area = x_train.loc[:, ['mean area', 'area error', 'worst area']]
x_train_smoothness = x_train.loc[:, ['mean smoothness', 'smoothness error', 'worst smoothness']]
x_train_compactness = x_train.loc[:, ['mean compactness', 'compactness error', 'worst compactness']]
x_train_concavity = x_train.loc[:, ['mean concavity', 'concavity error', 'worst concavity']]
x_train_cp = x_train.loc[:, ['mean concave points', 'concave points error', 'worst concave points']]
x_train_symmetry = x_train.loc[:, ['mean symmetry', 'symmetry error', 'worst symmetry']]
x_train_fd = x_train.loc[:, ['mean fractal dimension', 'fractal dimension error', 'worst fractal dimension']]

In [41]:
x_test_radius = x_test.loc[:, ['mean radius', 'radius error', 'worst radius']]
x_test_texture = x_test.loc[:, ['mean texture', 'texture error', 'worst texture']]
x_test_perimeter = x_test.loc[:, ['mean perimeter', 'perimeter error', 'worst perimeter']]
x_test_area = x_test.loc[:, ['mean area', 'area error', 'worst area']]
x_test_smoothness = x_test.loc[:, ['mean smoothness', 'smoothness error', 'worst smoothness']]
x_test_compactness = x_test.loc[:, ['mean compactness', 'compactness error', 'worst compactness']]
x_test_concavity = x_test.loc[:, ['mean concavity', 'concavity error', 'worst concavity']]
x_test_cp = x_test.loc[:, ['mean concave points', 'concave points error', 'worst concave points']]
x_test_symmetry = x_test.loc[:, ['mean symmetry', 'symmetry error', 'worst symmetry']]
x_test_fd = x_test.loc[:, ['mean fractal dimension', 'fractal dimension error', 'worst fractal dimension']]

## 모델링
![예시](https://raw.githubusercontent.com/DrKAI/image/main/cancer_locally.png)

1. 윗 단계에서 생성한 변수의 **형태**를 이용하여, 각 변수별 input layer를 만든다.
2. node가 12개인 hidden layer를 **각각** 생성하여, 각 input layer와 연결한다.
3. 생성된 hidden layer를 add 한다.
---
4. 전체 변수를 이용한 input layer를 만든다.
5. 3번 단계로 add 된 레이어와 4번 단계로 만들어진 input layer를 concat한다.
---
6. node가 128개인 hidden layer를 5번 단계로 concat된 레이어와 연결한다.
7. output layer를 생성하고 compile 한다.

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *

In [54]:
keras.backend.clear_session()

radius = Input(shape=(3,))
x1 = Dense(12, activation='relu')(radius)

texture = Input(shape=(3,))
x2 = Dense(12, activation='relu')(texture)

perimeter = Input(shape=(3,))
x3 = Dense(12, activation='relu')(perimeter)

area = Input(shape=(3,))
x4 = Dense(12, activation='relu')(area)

smoothness = Input(shape=(3,))
x5 = Dense(12, activation='relu')(smoothness)

compactness = Input(shape=(3,))
x6 = Dense(12, activation='relu')(compactness)

concavity = Input(shape=(3,))
x7 = Dense(12, activation='relu')(concavity)

cp = Input(shape=(3,))
x8 = Dense(12, activation='relu')(cp)

symmetry = Input(shape=(3,))
x9 = Dense(12, activation='relu')(symmetry)

fd = Input(shape=(3,))
x10 = Dense(12, activation='relu')(fd)

x = Add()([x1, x2, x3, x4, x5, x6, x7, x8, x9, x10])
inputs = Input(shape=(30, ))

x = Concatenate()([x, inputs])
x = Dense(128, activation='relu')(x)

outputs = Dense(1, activation='sigmoid')(x)

model = keras.models.Model([radius, texture, perimeter, area, smoothness, compactness, concavity, cp, symmetry, fd, inputs], outputs)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 3)]          0           []                               
                                                                                              

## 평가

1. 모델의 정확도를 확인하시오.
2. 악성이라 예측한 것 중 실제 악성으로 나온 비율을 구하시오. (precision)
3. 실제 악성 중 모델이 악성이라 예측한 것의 비율을 구하시오. (recall)
4. 정상이라 예측한 것 중 실제 정상으로 나온 비율을 구하시오. (precision)

In [51]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, restore_best_weights=True)

In [55]:
model.fit([x_train_radius, x_train_texture, x_train_perimeter, x_train_area, x_train_smoothness, x_train_compactness, x_train_concavity, x_train_cp, x_train_symmetry, x_train_fd, x_train], 
          y_train.astype(float), epochs=500, callbacks=[es], validation_split=.2)

Epoch 1/500
12/12 [==============================] - 2s 39ms/step - loss: 15.8918 - accuracy: 0.5027 - val_loss: 12.3884 - val_accuracy: 0.6484
Epoch 2/500
12/12 [==============================] - 0s 8ms/step - loss: 5.6051 - accuracy: 0.5467 - val_loss: 2.0857 - val_accuracy: 0.7582
Epoch 3/500
12/12 [==============================] - 0s 7ms/step - loss: 1.8954 - accuracy: 0.7363 - val_loss: 0.4402 - val_accuracy: 0.9121
Epoch 4/500
12/12 [==============================] - 0s 6ms/step - loss: 0.8102 - accuracy: 0.8654 - val_loss: 0.1985 - val_accuracy: 0.9451
Epoch 5/500
12/12 [==============================] - 0s 6ms/step - loss: 0.4133 - accuracy: 0.9038 - val_loss: 0.2926 - val_accuracy: 0.8901
Epoch 6/500
12/12 [==============================] - 0s 6ms/step - loss: 0.4234 - accuracy: 0.8984 - val_loss: 0.3809 - val_accuracy: 0.8462
Epoch 7/500
12/12 [==============================] - 0s 7ms/step - loss: 0.3843 - accuracy: 0.8984 - val_loss: 0.2109 - val_accuracy: 0.9451
Epoch 8/50

In [57]:
pred = model.predict([x_test_radius, x_test_texture, x_test_perimeter, x_test_area, x_test_smoothness, x_test_compactness, x_test_concavity, x_test_cp, x_test_symmetry, x_test_fd, x_test])

In [59]:
pred = np.where(pred> 0.5, 1, 0)

In [62]:
from sklearn.metrics import classification_report

print(classification_report(y_test.astype(float), pred.reshape(-1)))

              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90        68
         1.0       0.86      0.83      0.84        46

    accuracy                           0.88       114
   macro avg       0.87      0.87      0.87       114
weighted avg       0.88      0.88      0.88       114

